# Pregunta 2

### a) En primer lugar se crea el dataframe y se analizan sus registros.

In [75]:
import urllib
import pandas as pd
train_data_url = "http://www.inf.utfsm.cl/~jnancu/stanford-subset/polarity.train"
test_data_url = "http://www.inf.utfsm.cl/~jnancu/stanford-subset/polarity.dev"
train_data_f = urllib.urlretrieve(train_data_url, "train_data.csv")
test_data_f = urllib.urlretrieve(test_data_url, "test_data.csv")
ftr = open("train_data.csv", "r")
fts = open("test_data.csv", "r")
rows = [line.split(" ",1) for line in ftr.readlines()]
train_df = pd.DataFrame(rows, columns=['Sentiment','Text'])
train_df['Sentiment'] = pd.to_numeric(train_df['Sentiment'])
rows = [line.split(" ",1) for line in fts.readlines()]
test_df = pd.DataFrame(rows, columns=['Sentiment','Text'])
test_df['Sentiment'] = pd.to_numeric(test_df['Sentiment'])
print "Dimensiones training set: ", train_df.shape
print "Dimensiones testing set: ", test_df.shape
print "----------------------------------------------------------"
print  "Conteo de registros de cada clase en el training set"
print train_df['Sentiment'].value_counts()
print "----------------------------------------------------------"
print  "Conteo de registros de cada clase en el testing set"
print test_df['Sentiment'].value_counts()
#print train_df.head()
#print test_df.tail()

Dimensiones training set:  (3554, 2)
Dimensiones testing set:  (3554, 2)
----------------------------------------------------------
Conteo de registros de cada clase en el training set
-1    1784
 1    1770
Name: Sentiment, dtype: int64
----------------------------------------------------------
Conteo de registros de cada clase en el testing set
-1    1803
 1    1751
Name: Sentiment, dtype: int64


### b) En esta sección se pre procesa el texto mediante la utilización de las operaciones lower-casing, y stemming. Para ello se crea la función word_extractor.

In [76]:
import re, time
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize
from nltk.stem.porter import PorterStemmer
def word_extractor(text,stop = 1):
    #wordlemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text)#substitute multiple letter by two
    words = ""
    '''wordtokens = [ wordlemmatizer.lemmatize(word.lower()) \
                  for word in word_tokenize(text.decode('utf-8', 'ignore')) ]'''
    wordtokens = [ stemmer.stem(word.lower()) \
                  for word in word_tokenize(text.decode('utf-8', 'ignore')) ]
    for word in wordtokens:
        if stop==1:
            if word not in commonwords:
                words+=" "+word
        else:
            words+=" "+word
    return words
print word_extractor("I love to eat cake")
print word_extractor("I love eating cake")
print word_extractor("I loved eating the cake")
print word_extractor("I do not love eating cake")
print word_extractor("I dont love eating cake")
print word_extractor("I like learning in  university")
print word_extractor("The cats are playing with fishes")
print word_extractor("He believes in miracles")

 love eat cake
 love eat cake
 love eat cake
 love eat cake
 dont love eat cake
 like learn univers
 cat play fish
 believ miracl


En el caso anterior, el stemming lleva todas las palabras a su raíz sin importar el contexto en donde estas se encuentren. Stemming no puede discriminar, y solo se basa en ver las conjugaciones de las palabras de forma heurísticas, y dependiendo de la terminación es como decide a que raíz pertenece. Esto puede ocasionar problemas como por ejemplo con la palabra university, en donde el proceso de stemming la transforma a univers y puede ocasionar perdida de informacion ya que cambia el significado.

Si no se realiza stemming, las palabras quedarían conjugadas y para un procesador de texto o clasificador, no serían equivalentes aunque estén en un contexto similar. El problema es entonces que se generarían mas palabras, con lo cual aumenta el vocabulario y por ende la complejidad del dataset.

Con respecto a los "stop words", estos se definen en un conjunto de articulos, pronombres y preposiciones, y son quitados de las cadenas de texto, ya que son palabras muy comunes y que no aportan significado al texto, aunque en algunas aplicaciones si pueden llegar a ser utilizadas.

### c) En este caso se utiliza un lemmatizer para comparar con los resultados anteriores.

In [77]:
def word_extractor2(text, stop=1):
    wordlemmatizer = WordNetLemmatizer()
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text)#substitute multiple letter by two
    words = ""
    wordtokens = [ wordlemmatizer.lemmatize(word.lower()) \
                  for word in word_tokenize(text.decode('utf-8','ignore')) ]
    for word in wordtokens:
        if stop==1:
            if word not in commonwords:
                words+=" "+word
        else:
            words+=" "+word
    return words

print word_extractor2("I love to eat cake")
print word_extractor2("I love eating cake")
print word_extractor2("I loved eating the cake")
print word_extractor2("I do not love eating cake")
print word_extractor2("I dont love eating cake")
print word_extractor2("I like learning in  university")
print word_extractor2("The cats are playing with fishes")
print word_extractor2("He believes in miracles")

 love eat cake
 love eating cake
 loved eating cake
 love eating cake
 dont love eating cake
 like learning university
 cat playing fish
 belief miracle


Procesando las cadenas mediante el lemmatizer, se conserva muchas veces la conjugación de las palabras según el contexto, y también son llevadas a su tronco léxico. Por ejemplo las palabras loved y eating se conservan, y en este caso university se mantiene igualmente, no así en el proceso de stemming. Entonces la lematización es un proceso mucho más complejo, que no solo revisa cada palabra y su terminación, sino que además observa el contexto para realizar la agrupación de palabras similares.

### d) 

In [78]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
# Se crean las listas con los textos procesados
texts_train = [word_extractor2(text) for text in train_df.Text]
texts_test = [word_extractor2(text) for text in test_df.Text]
vectorizer = CountVectorizer(ngram_range=(1, 1), binary='False')
vectorizer.fit(np.asarray(texts_train))
#Se crea la matriz con la cuenta de los tokens(test y train)
features_train = vectorizer.transform(texts_train)
features_test = vectorizer.transform(texts_test)
print "Dimension conjunto de entrenamiento: ",features_train.shape
print "Dimension conjunto de pruebas: ", features_test.shape
print "--------------------------------------------------------------"
#label 0 y 1
labels_train = np.asarray((train_df.Sentiment.astype(float)+1)/2.0)
labels_test = np.asarray((test_df.Sentiment.astype(float)+1)/2.0)
vocab = vectorizer.get_feature_names()
dist_tr=list(np.array(features_train.sum(axis=0)).reshape(-1,))
dist_ts=list(np.array(features_test.sum(axis=0)).reshape(-1,))
#for tag, count in zip(vocab, dist):
#    print count, tag
index_tr= sorted(range(len(dist_tr)), key=lambda k: dist_tr[k], reverse=True )
index_ts= sorted(range(len(dist_ts)), key=lambda k: dist_ts[k], reverse=True )
#print sorted(dist_tr, reverse=True)[0]
print "10 Palabras mas frecuentes \n"
print "{0:<30}{1} \n".format("Conjunto de entrenamiento","Conjunto de pruebas")
for i in range (0,10):
    m1=index_tr[i]
    m2=index_ts[i]
    print '{0:<5}{1:<25}{2:<5}{3}'.format(dist_tr[m1], vocab[m1], dist_ts[m2], vocab[m2])

Dimension conjunto de entrenamiento:  (3554, 9663)
Dimension conjunto de pruebas:  (3554, 9663)
--------------------------------------------------------------
10 Palabras mas frecuentes 

Conjunto de entrenamiento     Conjunto de pruebas 

566  film                     558  film
481  movie                    540  movie
246  one                      250  one
245  like                     238  ha
224  ha                       230  like
183  make                     197  story
176  story                    175  character
163  character                165  time
145  comedy                   161  make
143  even                     134  comedy


Se aprecia en la tabla anterior, que el vocabulario se compone de 9663 palabras o tokens,  y que tanto los set de entrenamiento y prueba contienen 3554 registros o "reviews" de usuarios. Las 10 palabras más frecuentes se repiten en gran medida en ambos conjuntos y conservan un orden similar, esto refleja que los datos presentan estructuras similares.

### e)

In [79]:
from sklearn.metrics import classification_report
def score_the_model(model,x,y,xt,yt,text):
    acc_tr = model.score(x,y)
    acc_test = model.score(xt[:-1],yt[:-1])
    print "Training Accuracy %s: %f"%(text,acc_tr)
    print "Test Accuracy %s: %f"%(text,acc_test)
    print "Detailed Analysis Testing Results ..."
    print(classification_report(yt, model.predict(xt), target_names=['+','-']))

La función $\textit{classification_report}$  imprime 3 métricas las cuales son precision, recall y F1 score.
$\textit{Precision}$ se define como:

$$ precision = \frac{tp}{tp+fp}$$ 

Donde tp representa los verdaderos positivos y fp los falsos positivos. Intuitivamente es la habilidad del clasificador para no etiquetar como positivo una muestra que es negativa. Su mejor valor es 1 y su peor valor es 0.

Recall score se define como: 

$$ recall = \frac{tp}{tp+fp}$$ 

Donde tp representa los verdaderos positivos y fp los falsos negativos. Recall representa la habilidad del clasificador para encontrar todos los ejemplos positivos. Su mejor valor es 1 y su peor valor es 0.

Entonces $\textit{precision}$ representa la exactitud o calidad del clasificador y $\textit{recall}$ la medida de completitud o cantidad.

Finalmente, F1 es una medida que utiliza las dos métricas anteriores en un promedio ponderado. Su fórmula es: 

$$ f1 = 2* \frac{precision * recall}{precision+recall}$$


## f) Bernoulli Naive Bayes

El siguiente código crea los textos y features sin stop words y además otro utilizando stemming, para posteriormente ser analizados.

In [80]:
# El caso comun en el puntop (d) es lemmatizer filtrando stopwords
# Se crea training y testing set sin filtrar stopwords mediante lemmatizer.

texts_train_SW = [word_extractor2(text, 0) for text in train_df.Text]
texts_test_SW = [word_extractor2(text,0) for text in test_df.Text]
vectorizer = CountVectorizer(ngram_range=(1, 1), binary='False')
vectorizer.fit(np.asarray(texts_train_SW))
#Se crea la matriz con la cuenta de los tokens(test y train)
features_train_SW = vectorizer.transform(texts_train_SW)
features_test_SW = vectorizer.transform(texts_test_SW)

# Se crea training y testing set utilizando stemming

texts_train_stem = [word_extractor(text) for text in train_df.Text]
texts_test_stem = [word_extractor(text) for text in test_df.Text]
vectorizer = CountVectorizer(ngram_range=(1, 1), binary='False')
vectorizer.fit(np.asarray(texts_train_stem))
#Se crea la matriz con la cuenta de los tokens(test y train)
features_train_stem = vectorizer.transform(texts_train_stem)
#print features_train_stem.shape
features_test_stem = vectorizer.transform(texts_test_stem)

# Training y testing set utilizando stemming sin filtrar stop words
texts_train_stemSW = [word_extractor(text,0) for text in train_df.Text]
texts_test_stemSW = [word_extractor(text,0) for text in test_df.Text]
vectorizer = CountVectorizer(ngram_range=(1, 1), binary='False')
vectorizer.fit(np.asarray(texts_train_stemSW))
#Se crea la matriz con la cuenta de los tokens(test y train)
features_train_stemSW = vectorizer.transform(texts_train_stemSW)
#print features_train_stem.shape
features_test_stemSW = vectorizer.transform(texts_test_stemSW)


##### Modelo con lemmatizer y filtrando stop words Bernoulli NB

Para el caso de Bernoulli naive bayes los features deben tener valor 0 o 1, esto lo hace automáticamente BernoulliNB de la librería sklearn.

In [81]:
from sklearn.naive_bayes import BernoulliNB
import random
def do_NAIVE_BAYES(x,y,xt,yt):
    model = BernoulliNB()
    model = model.fit(x, y)
    score_the_model(model,x,y,xt,yt,"BernoulliNB")
    return model
model=do_NAIVE_BAYES(features_train,labels_train,features_test,labels_test)

Training Accuracy BernoulliNB: 0.958638
Test Accuracy BernoulliNB: 0.738531
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.75      0.73      0.74      1803
          -       0.73      0.75      0.74      1751

avg / total       0.74      0.74      0.74      3554



La tabla anterior muestra que el desempeño de Bernoulli Naive Bayes no es del todo bueno, ya que presenta una alta exactitud en el set de entrenamiento, pero mucho peor en el de pruebas, dando entender que el modelo esta sobre ajustando los datos. Esto también queda en evidencia con los otros indicadores como precision y recall, de donde la calidad para clasificar no es muy alta, y tampoco es completa. Esto puede deberse principalmente a que no se toma en cuenta la cantidad de palabras o tokens en cada registro, sino que son transformados a 0 o 1, perdiendo así información.

##### Modelo con lemmatizer sin filtrar stop words Bernoulli NB

In [82]:
model_SW=do_NAIVE_BAYES(features_train_SW,labels_train,features_test_SW,labels_test)

Training Accuracy BernoulliNB: 0.955262
Test Accuracy BernoulliNB: 0.748663
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.76      0.74      0.75      1803
          -       0.74      0.76      0.75      1751

avg / total       0.75      0.75      0.75      3554



Aquí no se presenta gran mejora, el modelo sigue sobre ajustando, pero al no considerarse el número de veces que aparece un stop word en un texto, sino que solo se interpreta como si está o no, esto no afecta demasiado a los resultados. Una ventaja es que requiere menos procesamiento de cómputo y se obtiene levemente un mejor resultado.

##### Modelo con stemmer Bernoulli NB

In [83]:
model_stem = do_NAIVE_BAYES(features_train_stem,labels_train,features_test_stem,labels_test)

Training Accuracy BernoulliNB: 0.942881
Test Accuracy BernoulliNB: 0.747819
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.76      0.74      0.75      1803
          -       0.74      0.75      0.75      1751

avg / total       0.75      0.75      0.75      3554



Al igual que el caso anterior, la mejora es muy pequeña, sin embargo al usar stemming el procesamiento de los datos es más rápido ya que no debe interpretar el contexto y además genera muchos menos features que lemmatization, 1700 palabras menos en el vocabulario aproximadamente. Esto muestra que bernoulli no se ve afectado por la cantidad de features en gran medida. Por lo mismo se concluye que stemming resulta ser mejor técnica de pre procesamiento que lemmatization en este caso.

##### Modelo con stemmer sin filtrar stop words Bernoulli NB

In [84]:
model_stemSW = do_NAIVE_BAYES(features_train_stemSW,labels_train,features_test_stemSW,labels_test)

Training Accuracy BernoulliNB: 0.938098
Test Accuracy BernoulliNB: 0.762173
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.77      0.76      0.77      1803
          -       0.76      0.76      0.76      1751

avg / total       0.76      0.76      0.76      3554



Este es el mejor caso de todos, corresponde a utilizar stemming y utilizar stop words. Este resultado intuitivamente parece incorrecto, ya que stemming pierde información, pero resulta ser que stemming en general presenta mejor desempeño que lemmatization. Por otra parte, al no filtrar los stop words, en textos relativamente pequeños, se presentan mejores resultados, esto posiblemente es porque al ser tan pocos conectores, pronombres, etc. en un texto pequeño, la información que adicionan al contexto del mensaje es beneficioso. No ocurre lo mismo en documentos muy grandes, en donde estos stop words comunes se suelen repetir cientos de veces.

##### Análisis exploratorio Bernoulli NB

Notar que en la impresión de la probabilidad por clase, la primera es la clase 0 o comentario negativo y la siguiente es la clase 1 o positivo.

In [85]:
test_pred = model.predict_proba(features_test)
#predictions = model.predict(features_test)
#print model.classes_
spl = random.sample(xrange(len(test_pred)), 5)
for text, sentiment, lab in zip(test_df.Text[spl], test_pred[spl], labels_test[spl]):
    print sentiment, text, "Valor real: ", lab, "\n"

[ 0.01108219  0.98891781] most thrillers send audiences out talking about specific scary scenes or startling moments ; " frailty " leaves us with the terrifying message that the real horror may be waiting for us at home .
Valor real:  1.0 

[ 0.99643381  0.00356619] opening with some contrived banter , cliches and some loose ends , the screenplay only comes into its own in the second half .
Valor real:  1.0 

[ 0.76991692  0.23008308] a muddy psychological thriller rife with miscalculations . it makes me say the obvious : abandon all hope of a good movie ye who enter here .
Valor real:  0.0 

[ 0.84304515  0.15695485] it believes it's revealing some great human truths , when , in reality , it's churning ground that has long passed the point of being fertile .
Valor real:  0.0 

[ 0.43784511  0.56215489] although some viewers will not be able to stomach so much tongue-in-cheek weirdness , those who do will have found a cult favorite to enjoy for a lifetime .
Valor real:  1.0 



Según lo apreciado, hay ciertas palabras que inmediatamente se categorizan como "negativas", por ejemplo las palabra worst o bad, o también existen algunas palabras que aumentan la probabilidad de ser clasificadas como positivos significativamente, como por ejemplo exciting. Esto se debe a que en muchas reseñas aparecen este tipo de palabras que demuestran expresión ya sea positiva o negativa.

Un problema que se presenta es cuando el clasificador no es capaz de reconocer la completitud del mensaje, por ejemplo este: "i simply can't recommend it enough .", le asigna 90% a que es negativo cuando realmente es positivo, entonces esto ocurre al no considerar el contexto entero del review, ya que Bernoulli se basa en la independencia condicional de las características.

# g) Multinomial Naive Bayes

##### Modelo con lemmatizer y filtrando stop words Multinomial NB

In [86]:
from sklearn.naive_bayes import MultinomialNB
import random
def do_MULTINOMIAL(x,y,xt,yt):
    model = MultinomialNB()
    model = model.fit(x, y)
    score_the_model(model,x,y,xt,yt,"MultinomialNB")
    return model
model=do_MULTINOMIAL(features_train,labels_train,features_test,labels_test)

Training Accuracy MultinomialNB: 0.959482
Test Accuracy MultinomialNB: 0.740782
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.75      0.73      0.74      1803
          -       0.73      0.75      0.74      1751

avg / total       0.74      0.74      0.74      3554



El modelo no es demasiado preciso, como ocurre con Bernoulli Naive Bayes, de hecho la accuracy obtenida en el set de pruebas es levemente superior a Bernoulli. Las métricas prácticamente son las mismas que Bernoulli NB, mostrando así que no existe mejoría entre una técnica u otra. Esto puede deberse en primera instancia a que los textos no presentan muchas palabras repetidas, luego la gran mayoría de los reviews tienen features binarios, con lo cual es similar a la distribución Binomial que consiste en "n" ensayos Bernoulli.

##### Modelo con lemmatizer sin filtrar stop words Multinomial NB

In [87]:
model_SW=do_MULTINOMIAL(features_train_SW,labels_train,features_test_SW,labels_test)

Training Accuracy MultinomialNB: 0.955543
Test Accuracy MultinomialNB: 0.747537
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.75      0.75      0.75      1803
          -       0.74      0.74      0.74      1751

avg / total       0.75      0.75      0.75      3554



Nuevamente, al no filtrar stop words se presenta un mejor resultado en la clasificación. Esto resulta ser raro, debido a que se esperaría que al eliminar la información no relevante el clasificador se comportara de mejor forma aumentando su desempeño. Por lo mismo se utiliza el mismo argumento que en Bernoulli NB, donde se asume que la longitud de los textos tienen pocas stop words y que estas ayudan a clasificar.

##### Modelo con stemmer Multinomial NB

In [88]:
model_stem = do_MULTINOMIAL(features_train_stem,labels_train,features_test_stem,labels_test)

Training Accuracy MultinomialNB: 0.942319
Test Accuracy MultinomialNB: 0.749789
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.76      0.75      0.75      1803
          -       0.74      0.75      0.75      1751

avg / total       0.75      0.75      0.75      3554



Utilizar un stemmer en Multinomial Naive Bayes resulta ser más beneficioso que un lemmatizer, ya que la accuracy sobre el set de pruebas aumenta. La precisión es mejor, al igual que el recall y el f1 score. Al parecer todo el análisis anterior indica que multinomial tiene el mismo comportamiento que bernoulli, es decir, es mejor utilizar stemming.

##### Modelo con stemmer sin filtrar stop words Multinomial NB

In [89]:
model_stemSW = do_MULTINOMIAL(features_train_stemSW,labels_train,features_test_stemSW,labels_test)

Training Accuracy MultinomialNB: 0.940630
Test Accuracy MultinomialNB: 0.759921
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.76      0.77      0.76      1803
          -       0.76      0.75      0.76      1751

avg / total       0.76      0.76      0.76      3554



Stemming sin filtro de stop words obtiene los mejores resultados para Multinomial NB, estableciendo así que stemming es mejor técnica de pre procesamiento de textos en este caso, y además al eliminar tokens no relevantes, requiere menor cantidad de recursos para procesar el modelo.

##### Analisis exploratorio Multinomial NB

In [90]:
test_pred = model.predict_proba(features_test)
#predictions = model.predict(features_test)
#print model.classes_
spl = random.sample(xrange(len(test_pred)), 5)
for text, sentiment, lab in zip(test_df.Text[spl], test_pred[spl], labels_test[spl]):
    print sentiment, text, "Valor real: ", lab, "\n"

[ 0.69665638  0.30334362] a showcase for both the scenic splendor of the mountains and for legendary actor michel serrault , the film is less successful on other levels .
Valor real:  0.0 

[ 0.18251059  0.81748941] the balkans provide the obstacle course for the love of a good woman .
Valor real:  0.0 

[ 0.08117932  0.91882068] . . . a guiltless film for nice evening out .
Valor real:  1.0 

[ 0.23291564  0.76708436] some movies blend together as they become distant memories . mention " solaris " five years from now and i'm sure those who saw it will have an opinion to share .
Valor real:  1.0 

[ 0.17848742  0.82151258] while certainly more naturalistic than its australian counterpart , amari's film falls short in building the drama of lilia's journey .
Valor real:  0.0 



# h) Regresión Logística regularizado.

A continuación se presenta la función que realiza la regresión logística regularizada, en donde se utilizan diversos parámetros de regularización. Notar que el parámetro "C" corresponde al inverso de la regularización, vale decir, mientras menor es su valor, mayor será la penalización. El significado de este parámetro, como ocurre en regresión lineal, es principalmente determinar que features, o tokens en este caso presentan mayor o menor significancia al modelo afectando directamente a los coeficientes de la regresión. Si no se aplica regularización, en modelos donde existen muchos features es probable que ocurra overfitting, ya que se necesitaría muchos datos de entrenamiento para que no ocurra esa situación.

El efecto esperado del parámetro "C" de la regresión, es que al disminuirlo, se espera mayor penalización, provocando menos overfitting, es decir generalizando en mayor medida ya que el modelo se ajusta a las variables que si son importantes, y así el problema no se vuelve demasiado especifico. Esto debe realizarse en clasificación de texto debido a que el vocabulario puede abarcar miles de palabras.

In [91]:
from sklearn.linear_model import LogisticRegression
def do_LOGIT(x,y,xt,yt):
    start_t = time.time()
    Cs = [0.01,0.1,10,100,1000]
    for C in Cs:
        print "Usando C= %f"%C
        model = LogisticRegression(penalty='l2',C=C)
        model = model.fit(x, y)
        score_the_model(model,x,y,xt,yt,"LOGISTIC")
        print "\n"
do_LOGIT(features_train,labels_train,features_test,labels_test)

Usando C= 0.010000
Training Accuracy LOGISTIC: 0.784468
Test Accuracy LOGISTIC: 0.678863
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.67      0.73      0.70      1803
          -       0.69      0.63      0.66      1751

avg / total       0.68      0.68      0.68      3554



Usando C= 0.100000
Training Accuracy LOGISTIC: 0.892234
Test Accuracy LOGISTIC: 0.719111
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.72      0.72      0.72      1803
          -       0.72      0.71      0.71      1751

avg / total       0.72      0.72      0.72      3554



Usando C= 10.000000
Training Accuracy LOGISTIC: 1.000000
Test Accuracy LOGISTIC: 0.718548
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.73      0.72      0.72      1803
          -       0.71      0.72      0.72      1751

avg / total       0.72 